In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [28]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Merge df_ratings and df_books on the 'isbn' column using a left join
df = df_ratings.merge(df_books, on="isbn", how = "left")

# user id's and rating counts
users = df["user"].value_counts()
# isbn's and rating counts
isbn = df["isbn"].value_counts()

# Filter users with at least 200 ratings
user_filter = users[users>=200].index
# Filter books with at least 100 ratings
isbn_filter = isbn[isbn>=100].index

# only keep rows that meet user_filter, and isbn_filter criteria
df_filtered = df.loc[(df["user"].isin(user_filter.values))&(df["isbn"].isin(isbn_filter.values))]

# make sure each user only has 1 rating for a given book
df_filtered = df_filtered.drop_duplicates(['title','user'])

# remove any rows that contain any missing values (NaNs) in their columns
df_filtered = df_filtered.dropna(how = 'any')

# create a new table from our filtered table with book titles as the index, user id's as the column, and values as rating values
# .fillna(0) will replace any missing values with 0
df_pivot = df_filtered.pivot(index = 'title', columns = 'user',values='rating' ).fillna(0)

# convert numpy array into CSR (compressed sparse row) matrix
df_matrix = csr_matrix(df_pivot.values)

# create a Nearest Neighbors model
# the cosine metric will be used to measure the distance between books
nn_model = NearestNeighbors(metric = 'cosine')

# train the model
nn_model.fit(df_matrix)

In [50]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommendation = [book,[]]
  # Find the nearest neighbors for the submitted book title, returns matrix of distances, and matrix of book indexes
  distances, books = nn_model.kneighbors([df_pivot.loc[book]],6,True)
  # reverse order of indices, and remove first element which should be the book itself
  books = df_pivot.iloc[np.flip(books[0])[:-1]].index.to_list()
  # reverse and remove first element from distances, which should correspond the the book removed
  distances = list(np.flip(distances[0])[:-1])

  # combine book informatin and distances into tuples
  for book_information in zip(books, distances):
    # convert each tuple back into a list item ["book info", distance] and append to second element in return list
    recommendation[1].append(list(book_information))
  return recommendation

In [51]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
